# ===========================================
# 1_data_exploration.ipynb
# Erste Analyse der Rohdaten
# ===========================================

# Zelle 1: Bibliotheken importieren
import os
import pandas as pd
import matplotlib.pyplot as plt
import re
import nltk
from nltk.tokenize import word_tokenize

# Sicherstellen, dass NLTK Tokenizer verfügbar ist
nltk.download('punkt')

# Pfad zu den Rohdaten (Annahme: Artikel oder Berichte im CSV-Format)
RAW_DATA_DIR = "../data/raw"
sample_file = os.path.join(RAW_DATA_DIR, "articles_raw.csv")  # passe den Dateinamen an

# ===========================================
# Zelle 2: Daten einlesen und Überblick gewinnen
# ===========================================
# Lese die CSV-Datei ein, die z.B. eine Spalte "text" enthält
df = pd.read_csv(sample_file)

print("Anzahl der Artikel:", len(df))
display(df.head())

# Zeige Spaltennamen und grundlegende Informationen
print("\nDataFrame Info:")
df.info()

print("\nAnzahl fehlender Werte pro Spalte:")
display(df.isnull().sum())

# ===========================================
# Zelle 3: Basisstatistiken zu den Texten
# ===========================================
# Berechne die Länge der Texte (z.B. in Zeichen und in Wörtern)
df["char_length"] = df["text"].astype(str).apply(len)
df["word_count"] = df["text"].astype(str).apply(lambda x: len(word_tokenize(x)))

print("\nStatistiken zur Zeichenzahl:")
display(df["char_length"].describe())

print("\nStatistiken zur Wortanzahl:")
display(df["word_count"].describe())

# ===========================================
# Zelle 4: Visualisierung der Textlängen
# ===========================================
plt.figure(figsize=(10, 4))
plt.hist(df["char_length"], bins=50, color="skyblue")
plt.xlabel("Anzahl Zeichen pro Artikel")
plt.ylabel("Anzahl Artikel")
plt.title("Verteilung der Artikellängen (Zeichen)")
plt.show()

plt.figure(figsize=(10, 4))
plt.hist(df["word_count"], bins=50, color="salmon")
plt.xlabel("Anzahl Wörter pro Artikel")
plt.ylabel("Anzahl Artikel")
plt.title("Verteilung der Artikellängen (Wörter)")
plt.show()

# ===========================================
# Zelle 5: Tokenstatistiken und häufige Wörter
# ===========================================
from collections import Counter

# Tokenisiere einen Beispieltext aus der ersten Zeile und zeige die ersten Tokens
sample_text = df["text"].iloc[0]
sample_tokens = word_tokenize(sample_text)
print("Erste 30 Tokens des ersten Artikels:")
print(sample_tokens[:30])

# Erstelle ein Frequenzverzeichnis über alle Artikel
all_tokens = []
for text in df["text"].dropna():
    tokens = word_tokenize(text)
    all_tokens.extend(tokens)

token_counts = Counter(all_tokens)
most_common = token_counts.most_common(30)

print("\nDie 30 häufigsten Tokens:")
for token, freq in most_common:
    print(f"{token}: {freq}")

# Balkendiagramm der häufigsten Tokens
tokens_labels, tokens_freq = zip(*most_common)
plt.figure(figsize=(12, 5))
plt.bar(tokens_labels, tokens_freq, color="mediumseagreen")
plt.xlabel("Tokens")
plt.ylabel("Häufigkeit")
plt.title("Top 30 häufigste Tokens in den Artikeln")
plt.xticks(rotation=45)
plt.show()

# ===========================================
# Zelle 6: Ausreißer und Beispiele
# ===========================================
# Finde die längsten und kürzesten Artikel (nach Wortanzahl)
long_texts = df.nlargest(3, "word_count")
short_texts = df.nsmallest(3, "word_count")

print("\nBeispiele für sehr lange Artikel:")
display(long_texts[["text", "word_count"]])

print("\nBeispiele für sehr kurze Artikel:")
display(short_texts[["text", "word_count"]])

# ===========================================
# Zelle 7: Fazit & Ausblick
# ===========================================
print("""
Fazit:
- Die Rohdaten umfassen Artikel mit einer breiten Verteilung in Länge.
- Es wurden grundlegende Statistiken (Zeichen- und Wortanzahl) sowie häufige Tokens ermittelt.
- Nächste Schritte: 
  - Daten bereinigen (z. B. Entfernen von Sonderzeichen, HTML-Tags)
  - Tokenisierung verfeinern
  - Aufteilung in Trainings-/Validierungs-/Testdaten (Splitting)
  - Erstellung eines Vokabulars für die Modellierung
  
Diese Erkenntnisse helfen dabei, die Anforderungen für das AutoSummary-Modell (LSTM-Seq2Seq oder Transformer) zu verstehen und die Daten optimal für das Training aufzubereiten.
""")
